# Problem 1 (12 points)

Recall the word ladder puzzle from Homework 1, in which vowel changes cost twice those of consonant changes. You implemented best-first search as a general procedure for solving this problem. Suppose now that we do not maintain a ```reached``` table and that children of expanded nodes are always added to the frontier.

1. Explain how this change affects the property of completeness. Remember that it is possible for a problem to have no solution. Consider each of the following algorithms: depth-first search, breadth-first search, uniform-cost search, and A* search with the simple Hamming distance heuristic.

2. Suppose that multiple solutions exist for a given problem. Without a reached table, which of the above four algorithms are guaranteed to return an optimal solution, and which are not? Explain for each of them.

3. Explain whether the "simple" Hamming distance and the Hamming distance with vowels heuristics are still admissible when A* is implemented without a reached table.

ENTER YOUR RESPONSES HERE

1. The reached table limits completeness, since it limits finding alternate solutions that use the same word. For DFS, the infinite nature of the problem means it will never be complete, but the reached table makes it less complete. For BFS, removing the reached table makes the search complete since it systematically goes throught each node in the tree. For UCS, removing the reached table makes it complete since it visits every node systematically. For an A* search with a Hamming distance heuristic, removing the reached table makes it complete since it also would search through all the nodes in the tree in a systematic way similar to UCS.

2. DFS is not guaranteed to return the optimal solution since it goes down one branch and returns the first solution seen. Since it only minimizes distance to the root node, BFS is only guaranteed to return the optimal solution if costs are uniform, which they are not in this case, so BFS is not guaranteed to return the optimal solution. UCS is is guaranteed to return the optimal solution since it checks nodes in order of minimal cost and returns the first one seen. A* search is only optimal if the heuristic function is admissable. Since the Hamming distance heuristic is admissable, A* is optimal.

3. Yes, they are still admissable because the reached table does not affect admissibility. Hamming distance is admissable since the total cost will always be greater than or equal to the number of indices where the corresponding letters are different. The Hamming distance with vowels is even more accurate, since it always is equal to the true cost. The second heuristic dominates the first since its value is always greater than or equal to the firsts.



# Problem 2 (16 points)

We are computing the value $v_0$ of a chance node $C_0$ in a game tree. $C_0$ has children $C_1, C_2, ..., C_n$, occurring with probability $p_1, p_2, ..., p_n$, respectively. We have already retrieved the values of the first $k$ children: $v_1, ..., v_k$. We have not yet seen the values of the remaining children nodes. Give an expression for the maximum possible value $v_0$, if the maximum possible value for any child node is $v_{\text{max}}$.

ENTER YOUR EXPRESSION HERE

(v1 * p1) + (v2 * p2) + ... + (vk * pk) + (vmax  * pk+1) + (vmax * pk+2) + ... + (vmax * pn)

X and O are playing 3x3 tic-tac-toe. Instead of playing optimally, O now plays randomly, choosing its move according to known probabilities each turn. X's strategy is implemented in an ```X_value``` function (signature shown below). You also have access to functions returning O's ```actions``` in a given state and the ```result``` of taking an action from a given state.

```
def X_value(state):
  INPUTS: A game state
  OUTPUT: Value of state assuming it is X's turn to move

def actions(state):
  INPUT: A game state during O's turn
  OUTPUTS: List of (action, probability) tuples

def result(state, action):
  INPUTS: A game state and action
  OUTPUT: New game state as a result of O taking action from state
```

Calling the above functions as necessary, complete the ```O_value``` function below, which returns the value of the given game state assuming it is O's turn to move. As the function loops through the children states to compute the expected value, it should also prune when possible. ```O_value``` should stop retrieving the values of its remaining children nodes as soon as it recognizes that it is no longer possible to return a value greater than ```alpha``` (it can simply return the value computed so far).

As a hint, you should be using the expression you derived above. You may assume that ```V_MAX``` is a known upper bound on state utility values and accessible in your implementation.

In [66]:
def O_value(state, alpha):
  isTerminal, score = terminal(state)
  if isTerminal:
    return score

  # YOUR CODE HERE

  expected_val = 0
  total_prob = 0

  o_actions = actions(state)

  for tup in o_actions:
    total_prob += tup[1]

    expected_val += tup[1] * X_value( result( state, tup[0] ) )

    if expected_val + (1-total_prob) * V_MAX <= alpha:
      break

  return expected_val


You can check that your implementation above is free of syntactical and logical errors by running the unit test below (note this does *not* give any indication of the correctness of your implementation). We cannot answer questions on what the expected output should be.

In [67]:
def terminal(s):
  return False, None
def actions(s):
  return [('a1',0.5), ('a2',0.5)]
def result(s,a):
  if a == 'a1': return 's1'
  elif a == 'a2': return 's2'
def X_value(s):
  if s == 's1': return 1
  elif s == 's2': return 1

V_MAX = 1
print(O_value('s0',1))

0.5


# Problem 3 (20 points)

Recall that $TD(0)$ solves the prediction problem of evaluating a fixed policy $\pi$ using temporal-difference updates to estimated state values. The update to the value $V^\pi(s)$ is given by

$$ V^\pi(s) \leftarrow V^\pi(s) + \alpha (r + \gamma V^\pi(s') - V^\pi(s)), $$

where $r$ is the observed reward, $s'$ is the successor state, $\alpha$ is the learning rate, and $\gamma$ is the discount factor.

Suppose that our agent has traversed a sequence of states $s_1, s_2, ..., s_n$ (following $\pi$) and observed the corresponding rewards $r_1, r_2, ..., r_{n-1}$. All distinct states in the problem have been encountered at least once prior to $s_n$. The underlying transition and reward functions map $(s,a,s')$ inputs to fixed probabilities and real values, respectively. 

1. Provide a set of equations in terms of the observed states, rewards, and problem parameters that, if solvable, produces the values to all states $V^\pi(s)$ such that all TD updates using the observed state-reward sequences are zero. It is acceptable for your set to contain possibly redundant equations.

2. How many unknowns are there in your equations? Explain whether it is possible to have more *unique* equations than unknowns and why that may occur.



ENTER YOUR RESPONSES HERE

1. 0 = r1 + gamma * V(s2) - V(s1),
0 = r2 + gamma * V(s3) - V(s2),
... ,
0 = rn-1 + gamma * V(sn) - V(sn-1)


2. There are n unknowns in the equations, each being a value we are trying to solve. It is possible to have more unknowns than equations since each equation represents a transition between states which is fewer than the actual number of states.

As with policy evaluation using a known model, an alternative to solving the equations that you wrote is dynamic programming, this time using the TD update rather than Bellman update. Starting with an initialized dictionary of values ```V``` of the form ```{state:value}```, this scheme should sweep over the lists of observed ```states``` and ```rewards``` sequences, performing a TD update to  ```V``` for each state-reward pair seen. This sweep should then be repeated until the maximum absolute change for a value is smaller than the provided ```threshold```. (You may assume that the provided inputs will produce a solution that converges.)

Implement the described algorithm in ```TD0``` below. In addition to the described inputs, note that we also have the ```alpha``` and ```gamma``` parameters.

In [68]:
def TD0(V, states, rewards, alpha, gamma, threshold):
  max_diff = float("inf")
  while max_diff >= threshold:
    # YOUR CODE HERE 
    temp_max = -1

    for x in range(len(rewards)):
      error = rewards[x] + gamma * V[states[x+1]] - V[states[x]]
      V[states[x]] = V[states[x]] + alpha * error

      if error > temp_max:
        temp_max = error

    max_diff = temp_max

  return V

You can check that your implementation above is free of syntactical and logical errors by running the unit test below (note this does *not* give any indication of the correctness of your implementation). We cannot answer questions on what the expected output should be.

In [69]:
states = ['s1', 's2', 's1']
rewards = [1, 1]
alpha = 0.5
gamma = 0.8
threshold = 1e-6

V = {'s1':0, 's2':0}
print(TD0(V, states, rewards, alpha, gamma, threshold))

{'s1': 4.999996728139351, 's2': 4.9999969413111405}


# Problem 4 (16 points)

We have a Naive Bayes model with class variable $Y$ and feature variables $F_1, ..., F_n$. Suppose we observe feature $F_e = f_e$. 

1.  Give an expression for $\Pr(F_q \mid f_e)$, the distribution of the query feature $F_q$ given the evidence feature $f_e$. You may also provide your answer in the form of an unnormalized distribution. All quantities in your expression should be contained in the set of Naive Bayes parameters.

2.  Briefly describe how you would estimate $\Pr(F_q \mid f_e)$ using likelihood weighting instead. What variables should be sampled, what order are they sampled in, and what sampling distributions are used? Also be sure to describe how the desired distribution is obtained from samples.

ENTER YOUR RESPONSES HERE

1.  Sum_y ( Pr(fe | y) * Pr(y) * Pr(Fq | y) )

2.  I would sample Y and Fq while fixing evidence fe. I would sample in topological order. The sampling distributions used are the known distribution which we sample from and the unknown target distribution which we are trying to find out. The desired distribution is obtained from the samples by weighting each sample based on the probability of evidence given its parents.

Use the expression that you came up with in 1. to implement ```feature_likelihood``` below. The function inputs are as follows:
*   ```prior``` is a 1D numpy array containing the distribution $\Pr(Y)$.
*   ```fe_given_y``` is a 1D numpy array containing the probabilities $\Pr(f_e \mid Y)$.
*   ```fq_given_y``` is a 2D numpy array, in which ```fq_given_y[i,j]``` is the probability $\Pr(F_q = i \mid Y = j)$.

Make sure that your outputs are properly normalized.

In [70]:
import numpy as np

def feature_likelihood(prior, fe_given_y, fq_given_y):
  # YOUR CODE HERE

  dists = list()

  for i in range(len(fq_given_y)):
    dists.append(prior * fe_given_y * fq_given_y[i])

  for x in range(len(dists)):
    sum = np.sum(dists[x])
    dists[x] = dists[x] / sum

  return dists


You can check that your implementation above is free of syntactical and logical errors by running the unit test below (note this does *not* give any indication of the correctness of your implementation). We cannot answer questions on what the expected output should be.

In [71]:
prior = np.array([0.5, 0.5])
fe_given_y = np.array([0.7, 0.6])
fq_given_y = np.array([[0.3, 0.6], [0, 0.4], [0.7, 0]])

print(feature_likelihood(prior, fe_given_y, fq_given_y))

[array([0.36842105, 0.63157895]), array([0., 1.]), array([1., 0.])]
